# Retrieval Augmented Generation

Ten notatnik pomoże Ci zapoznać się z podstawowym podejściem do Retrieval Augmented Generation (RAG). W trakcie ćwiczenia będziemy korzystać głównie z bibliotek [google-generativeai](https://github.com/google-gemini/generative-ai-python), [langchain](https://python.langchain.com/) i [trulens](https://www.trulens.org/). Po uzupełnieniu tego notatnika powinieneś wiedzieć:
- czym jest RAG,
- jak z poziomu kodu komunikować się z LLMem,
- jak pobierać i dzielić dokumenty tekstowe na potrzeby RAG,
- jak policzyć zanurzenia i przechowywać zanurzenia dla swoich dokumentów,
- jak wykonać sematyczne wyszukiwanie wśród dokumentów,
- jak wstrzykiwać kontekst do zapytań do LLMów,
- jak prowadzić dłuższe konwersacje z LLMem,
- jak mierzyć i monitorować jakość odpowiedzi LLMów.


## Przygotowanie

Do pracy z LLMami potrzebny będzie klucz API do dostawcy takowych. Na tych zajęciach będziemy łączyć się z modelami firmy Google. Jeśli jeszcze nie masz klucza API, skorzystaj z instrukcji na stronie: https://ai.google.dev/gemini-api/docs/api-key.

Oprócz tego będziemy potrzebować kilku bibliotek:

```{command}
pip install google-generativeai langchain langchain-community langchain-google-genai pypdf youtube-transcript-api chromadb trulens trulens-apps-langchain trulens-providers-langchain
```


In [ ]:
!pip install google-generativeai langchain langchain-community langchain-google-genai pypdf youtube-transcript-api chromadb trulens trulens-apps-langchain trulens-providers-langchain

Jeśli masz potrzebne biblioteki, możesz sprawdzić czy jesteś w stanie komunikować się z LLMem. Wykonaj poniższy kod, żeby sprawdzić czy wszystko działa. Jeśli nie masz klucza API w zmiennej systemowej, po prostu wklej go do kodu poniżej zamiast odwołania do zmiennej systemowej. Jeśli chcesz pracować ze zmienną systemową, prawdopodobnie będziesz musiał po dodaniu zmiennej systemowej uruchomić nowy terminal, w którym nowa zmienna będzie widoczna, ponownie odpalić serwer jupyter.

In [ ]:
import os
import google.generativeai as genai
import numpy as np
api_key = ...

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = api_key

**Zad. 1: Uruchom poniższy kod. Możesz zmodyfikować opis systemu i zapytanie.**

In [ ]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction="You are a poetic assistant, skilled in songwriting."
    )
response = model.generate_content("Write lyrics for a country song about Polish winters.")
print(response.text)

In [ ]:
print(response)

Oprócz informacji zwrotnej od LLMa mamy również szereg metadanych, w tym ile tokenów wysłaliśmy a ile odebraliśmy.

Na koniec zapiszemy sobie treść wyniku, może się jeszcze przyda.

In [ ]:
song = response.text

## Wgrywanie danych

Langchain (jak i inne narzędzia, np. LlamaIndex) posiadają szereg funkcji pomocniczych do zdobywania tekstu z filmów, podkastów, baz danych, stron internetowych i innych źródeł. Na potrzeby tych zajęć, jako kontekst wykorzystamy dwie krótkie książki Andrew Ng w formacie PDF i napisy z dwóch filmów na Youtube.

In [ ]:
# Wczytujemy książki
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("./books")
docs = loader.load()

PDFy są domyślnie dzielone na strony, zatem zmienna `docs` to lista stron. Zobaczmy ile stron mają łącznie obie książki i co jest na stronie 23.

In [ ]:
print(f"Łacznie w książka jest {len(docs)} stron.")
print()
print(f"Zawartość 23. strony w kolekcji to:\n{docs[22].page_content}")

Teraz pobierzemy napisy z dwóch filmików gdzie przemawia Andrew Ng. Zwróć uwagę na parametr language - pozwala nam on priorytetyzować ręcznie sporządzone napisy dostarczone przez twórcę filmu (automatycznie wygenerowane napisy w przypadku pierwszego filmu nie są idealne...).

In [ ]:
from langchain_community.document_loaders import YoutubeLoader
clips = []

for link in ["https://www.youtube.com/watch?v=5p248yoa3oE",
             "https://www.youtube.com/watch?v=0jspaMLxBig"]:
    loader = YoutubeLoader.from_youtube_url(link, add_video_info=False, language=["en-US", "en-GB", "en"])
    clips.extend(loader.load())

**Zad. 2: Zobacz ile fragmentów (obiektów typu Document) mają łącznie teksty obu filmików i co jest w transkrypcie o indeksie 0.**

Na koniec rozszerzemy tekst z książek o transkrypty z filmów, żeby mieć jedną wspólną listę kontekstów.

In [ ]:
docs.extend(clips)
print(len(docs))

## Dzielenie tekstu na części

Na temat dzielenia tekstu na mniejsze fragmenty można by przygotować osobny tutorial. Można dzielić na podstawie znaków, tokenów, parsować zdania za pomocą nltk, wykrywać akapity i rozdziały, tworzyć hierarchie fragmentów. Przykłady bardziej zaawansowanych technik z wykorzystaniem LlamaIndex można znaleźć na [blogach](https://blog.llamaindex.ai/a-cheat-sheet-and-some-recipes-for-building-advanced-rag-803a9d94c41b) i [darmowych kursach](https://www.deeplearning.ai/short-courses/building-evaluating-advanced-rag/).

W ramach tych zajęć skorzystamy tylko z jednego prostego podejścia do dzielenia tekstu opartego na wybranych znakach. `RecursiveCharacterTextSplitter`. bo tak nazywa się klasa z której skorzystamy, stara się dzielić tekst na mniejsze części o zadanej długości. W tym celu wyszukuje wskazanych znaków i wybiera pierwszy, który pozwoli uzyskać fragment nie dłuższy niż wskazana liczba znaków. Zobacz jak to działa na przykładzie poniżej.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", ", ", " ", ""]
)
r_splitter.split_text(song)

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=10,
    separators=["\n\n", "\n", "\. ", ", ", " ", ""]
)
r_splitter.split_text(song)

**Zad. 3: Podziel dokumenty w zmiennej `docs` na części o długości 750 z zakładką o rozmiarze 150. Możesz do tego użyć fukcji `split_documents` zamiast `split_text`. Wynik przypisz do zmiennej `splits`. Sprawdź ile fragmentów zawiera `splits` i porównaj to z liczbą elementów w `docs`.**

## Tworzenie i przechowywanie zanurzeń

Jak wiadomo z wykładów, tekst można zapisywać do baz w różnorakich formatach, jednak dominują formaty wektorowe. Pracując z LLMami będzie nam zależeć na wyszukiwaniu semantycznym, czyli opratnym na znaczeniu tekstu a nie na występowaniu konkretnych słów. Użyjemy zanurzeń od Google, ale można równie dobrze korzystać z zanurzeń udostępnianych na HuggingFace (no. BAAI/bge-small-en-v1.5 albo BAAI/bge-large-en-v1.5) i liczyć je lokalnie na komputerze.

Wykonaj poniższy kod, aby policzyć zanurzenia dla przykładów z wykładu.

In [ ]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
embedding = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001"
    )

sentence1 = "Ja lubię eksplorację danych."
sentence2 = "Ja lubię pływać."
sentence3 = "Ja uwielbiam biegać."

embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

**Zad. 4: Podejrzyj jak wygląda takie zanurzenie i jaką ma długość. Następnie policz odległość między każdą parą zanurzeń. Czy zanurzenia 2 i 3 są do siebie bardziej podobne niż pozostałe pary?**

In [ ]:
print(len(embedding2))

print(np.dot(embedding1, embedding2))
print(np.dot(embedding1, embedding3))
print(np.dot(embedding2, embedding3))

Gdy już wiemy jak działa zanurzenia i mamy odpowiedni obiekt do tworzenia takowych w zmiennej `embedding`, czas stworzyć bazę danych dla naszych tekstów. Opcji jest wiele, ale my skorzystamy z bazy Chroma, która potrafi działać w pamięci jak i szybko stworzyć małą bazę sqlite lokalnie na dysku.

Uruchom poniższy kod, aby stworzyć bazę zanurzeń, zapisać ją na dysk i zobaczyć ile elementów ma w środku.

In [ ]:
from langchain.vectorstores import Chroma

#!rm -rf ./chroma
persist_directory = './chroma/'

vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

Po stworzeniu bazy możemy wypróbować wspomniane wyszukiwanie semantyczne. Poniżej napiszemy treść przykładowego zapytania i poprosimy o 3 najbardziej pasujące fragmenty z bazy.

In [ ]:
question = "What is an eyeball dataset?"
relevant_splits = vectordb.similarity_search(question, k=3)

**Zad. 5: Sprawdź ile fragmentów zwróciła baza. Co jest w tych fragmentach? Z których książek lub filmów one pochodzą?**

Powyżej zapytanie semantyczne zwróciło obiecujące wyniki. Czasami niestety to nie wystarcza. W przypadku duplikatów w bazie, trzeba dbać o różnorodność zwracanych fragmentów. W innych przypadkach trzeba oprócz wyszukiwania semantycznego ograniczyć się do wybranych dokumentów bo są one wprost wskazane w pytaniu. Tymi rzeczami nie mamy czasu się zajmować, ale w zależności od potrzeb można takie problemy rozwiązywać odrobiną technik z tradycyjnych baz danych lub poprosić LLM o pomoc przy zapytaniu do bazy kontekstów.

## Odpowiadanie na pytania z wykorzystaniem kontekstu (RAG)

Po zebraniu dokumentów, podzieleniu ich na mniejsze fragmenty, policzeniu zanurzeń i zapisaniu ich do bazy, możemy przejść do RAG. To jest moment w, którym błyszczy langchain. Langchain pozwala tworzyć strumienie wywołań różnych narzędzi i przekazywać wyniki jednego narzędzia jako wejście do innego. W tym wypadku przekażemy zapytanie do bazy zanurzeń a następnie zapytanie wraz fragmentami z bazy przekażemy do LLMa. Taki prosty łańcuch wywołań można stworzyć korzystając z kodu poniżej.

In [ ]:
from langchain.chains import RetrievalQA
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
import warnings
warnings.simplefilter("ignore") # API zmienia się bardzo szybko i co rusz coś staje się deprecated. Wyciszymy ostrzeżenia.

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,  #Niska temperatura = mało losowości w odpowiedzi LLMa
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

Mając taki prosty łąncuch wywołań możemy zadać zapytanie do LLMa licząc, że skorzysta z wyszukanego tekstu podczas udzielania odpowiedzi.

In [ ]:
question = "What is an eyeball dataset?"

result = qa_chain({"query": question})
print(result["result"])

_Ciekawscy mogą teraz uruchomić Gemini w osobnym oknie i zobaczyć jak LLM odpowiedziałby na to samo pytanie bez znajomości kontekstu._

Rzeczy, które mogą się wydarzyć jest znacznie więcej. Jedną z istotniejszych jest dodanie zdefiniowanego przez nas prompta. Prompt engineering jest sztuką, która potrafi znacząco wpłynąć na działanie RAG. Niekiedy prompty są baaaardzo długie, aby skutecznie nakierować LLM na to o co deweloperowi chodzi. Poniżej, poprosimy o to żeby odpowiedzi były zwięzłe i żeby zawsze kończyły się frazą "Thanks for asking!". Ponadto poprosimy o zwracanie dokumentów kontekstowych fraz z odpowiedzią.

In [ ]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

**Zad. 6: Ponownie zadaj to samo zapytanie. Jak zmieniła się odpowiedź? Zobacz z jakich książek i stron pochodzi kontekst.**

Odpowiedź w istocie jest zwięzła. Poprośmy żeby ją rozwinął.

In [ ]:
result = qa_chain({"query": "Can you provide a a longer and more detailed response to my last question?"})
print(result["result"])

Sprawdźmy skąd pochodzi kontekst tej odpowiedzi.

In [ ]:
print(result["source_documents"][0].metadata)
print(result["source_documents"][1].metadata)
print(result["source_documents"][2].metadata)

Odpowiedź wynika stąd, że w obecnej formie nasz RAG nie ma pamięci. Każde zapytanie jest niezależne i nie możemy prowadzić dyskusji pogłębiającej poprzednie pytania. Zaraz to naprawimy.

## Pamięć i czat

Jeśli zależy nam na dłuższych, wieloetpaowych rozmowach z LLMem, będziemy potrzebować pamięci. Pamięc będzie po prostu zapamiętywać zadane pytania i uzyskane odpowiedzi. Poniżej kod tworzący pamięć.

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

memory.clear() # gdy chcemy zresetować pamięć

Teraz zmienimy łańcuch wywołań na taki korzystający z pamięci. Uwaga, ten chain ma trochę inne nazwy pól zapytania i odpowiedzi.

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
chat_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

Teraz ponówmy nasz eksperyment. Zadajmy pytanie i poprośmy o rozszerzenie poprzedniej odpowiedzi.

In [ ]:
question = "What is an eyeball dataset?"

result = chat_chain({"question": question})
print(result["answer"])

In [ ]:
result = chat_chain({"question": "Can you provide a longer and more detailed response to my last question?"})
print(result["answer"])

## Ocena systemu

Jako ostatni element zapoznamy się z metodami oceny systemów typu RAG. Twórcę takiego systemu może interesować na ile trafne są odpowiedzi, na ile kontekst wspiera odpowiedź i na ile kontekst pasuje do pytania. Te trzy elementy sprawdzają miary Answer Relevance, Groundedness i Context Relevance. Poniżej kod tworzący nowy chain (taki kóry pozwoli zajrzeć do kontekstu i sklei nam wszystkie fragmentu kontekstu w jeden łańcuch znaków. Następnie definicja wspomnienych trzech miar.

In [ ]:
from trulens.core import Feedback, TruSession

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

tru = TruSession()
tru.reset_database()

In [ ]:
from trulens.providers.langchain import Langchain
from trulens.apps.langchain import TruChain
from trulens.apps.langchain import WithFeedbackFilterDocuments

from langchain_google_genai.llms import GoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
import numpy as np

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

gemini = GoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
)
provider = Langchain(chain=gemini)

context = TruChain.select_context(rag_chain)


# Groundedness
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(context.collect())  # collect context chunks into a list
    .on_output()
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = Feedback(
    provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()
# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

Teraz stworzymy obiekt `tru_recorder`, który będzie monitorował wszystkie zapytania i je oceniał. Zamiast testować na jednym zapytaniu przeprowadzimy eksperyment i policzymy średnią z 5 zapytań żeby ocenić nasz system.

In [ ]:
from trulens.apps.langchain import TruChain
from time import sleep

def invoke_with_retries(rag_chain, question, max_retries=5): #we are likely to exceed rate limit, that is why we use this function - if the error still occurs limit the number of eval questions
    retries = 0
    while retries < max_retries:
        try:
            with tru_recorder as recording:
                return rag_chain.invoke(question)
        except Exception as e:
            if "429" in str(e):
                retries += 1
                wait_time = 2 ** retries  # Exponential backoff
                print(f"Rate limit hit. Retrying in {wait_time} seconds...")
                sleep(wait_time)
            else:
                raise e
    raise Exception("Max retries exceeded")

tru_recorder = TruChain(
    rag_chain,
    app_name="ChatApplication",
    app_version="Chain1",
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness],
)

eval_questions = [
    'What is an eyeball dataset?',
    'What are the keys to building a career in AI?',
    'What is the importance of networking in AI?',
    'How can altruism be beneficial in building a career?',
    'What is imposter syndrome and how does it relate to AI?'
]

for question in eval_questions:
  llm_response = invoke_with_retries(rag_chain, question, max_retries=6)


In [ ]:
tru.get_leaderboard()

Możemy zobaczyć jak nasz system sobie radzi uruchamiając komendę `tru.get_leaderboard()`. Jeśli będziemy testować wiele wersji aplikacji (parametr `app_id`) to możemy porównywać wersje między sobą właśnie w ramach tabeli wyników.

Na deser: poniższy kod uruchamia dashboard gdzie można przeanalizować każde zapytanie i uzyskane miary.

In [ ]:
tru.run_dashboard() #go to records

**Zad. 7: Czy jesteś w stanie stworzyć nową wersję RAG, która uzyska lepsze metryki? Spróbuj zmienić prompt, żeby odpowiedzi od LLMa były dłuższe. Spróbuj zmienić liczbę fragmentów wydobywanych z bazy. Spróbuj zmienić długość fragmentów, na które dzielone są dokumenty.**